In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.externals import joblib
from bayes_opt import BayesianOptimization
from gensim import models


In [ ]:
max_length = 500  # 表示样本表示最大的长度,表示降维之后的维度
sentence_max_length = 1500  # 表示句子/样本在降维之前的维度
Train_features3, Test_features3, Train_label3, Test_label3 = [], [], [], []

# 读取训练好的词嵌入向量和文本文件

In [ ]:
# 通过models.KeyedVectors加载预训练好的embedding
fast_embedding = models.FastText.load('/home/user10000411/notespace/Embedding/models/fast_model')
w2v_embedding = models.Word2Vec.load('/home/user10000411/notespace/Embedding/models/w2v_model_50000')

print("fast_embedding输出词表的个数{},w2v_embedding输出词表的个数{}".format(
    len(fast_embedding.wv.vocab.keys()), len(w2v_embedding.wv.vocab.keys())))

print("取词向量成功")

In [ ]:
# 读取train_clean.tsv ，test_clean.tsv训练集和测试集文件
# hint: 通过pandas中的read_csv读取数据集
train = pd.read_csv('/home/user10000411/dataset/图书分类文本数据集/train_clean.csv', sep='\t')
test = pd.read_csv('/home/user10000411/dataset/图书分类文本数据集/test_clean.csv', sep='\t')
print("读取数据完成")

将df中的label映射为数字标签并保存到labelIndex列中

In [ ]:
labelName = train.label.unique()  # 全部label列表
labelIndex = list(range(len(labelName)))  # 全部label标签
labelNameToIndex = dict(zip(labelName, labelIndex))  # label的名字对应标签的字典
labelIndexToName = dict(zip(labelIndex, labelName))  # label的标签对应名字的字典
train["labelIndex"] = train.label.map(labelNameToIndex)
test["labelIndex"] = test.label.map(labelNameToIndex)

In [ ]:
print(test["labelIndex"])

In [ ]:
def query_cut(query):
    '''
    函数说明：该函数用于对输入的语句（query）按照空格进行切分
    '''
    # 第一步：定义一个query_cut函数 将query按空格划分并返回，
    query_list = query.split(' ')
    return query_list

# 第二步：然后train和test中的每一个样本都调用该函数，
# 将划分好的样本分别存储到train["queryCut"]和test["queryCut"]中

train["queryCut"] = [query_cut(_) for _ in train.text]
test["queryCut"] = [query_cut(_) for _ in test.text]
print("切分数据完成")

In [ ]:
with open('stopwords.txt', "r") as f:
    # 第一步：按行读取停用词文件
    stopWords = []
    for line in f:
        stopWords.append(line.strip('\n'))

def rm_stop_word(wordList):
    '''
    函数说明：该函数用于去除输入样本中的存在的停用词
    Return: 返回去除停用词之后的样本
    '''
    # 第二步：去除每个样本中的停用词并返回新的样本
    forRemove = []
    for word in wordList:
        if word not in stopWords:
            forRemove.append(word)            
    
    return forRemove
    
train["queryCutRMStopWord"] = train["queryCut"].apply(rm_stop_word)
# dev["queryCutRMStopWord"] = dev["text"].apply(rm_stop_word)
test["queryCutRMStopWord"] = test["queryCut"].apply(rm_stop_word)
print("去除停用词")
print(type(train["queryCutRMStopWord"]))

In [ ]:
def softmax_cal(x):#对于矩阵的softmax, 输出与输入同形状
    tmp = np.max(x,axis=1) # 得到每行的最大值，用于缩放每行的元素，避免溢出
    x -= tmp.reshape((x.shape[0],1)) # 利用性质缩放元素
    x = np.exp(x) # 计算所有值的指数
    tmp = np.sum(x, axis = 1) # 每行求和        
    x /= tmp.reshape((x.shape[0], 1)) # 求softmax
    return x

In [ ]:
def store(em, n): # 用于滑窗
    wn = em.shape[0]
    vn = em.shape[1]
    temp = np.empty((wn-n+1,vn))
   
    for i in range(wn-n+1):
        temp[i]=np.mean(em[i:i+n,:],axis=0)
        
    max_line = np.mean(temp, axis=0)
    
    return max_line

In [ ]:
def cal_cos(x,y):
    v = np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))
    return v

In [ ]:
def Find_embedding_with_windows(embedding_matrix):
    '''
    函数说明：该函数用于获取在大小不同的滑动窗口(k=[2, 3, 4])， 然后进行平均或取最大操作。
    参数说明：
        - embedding_matrix：样本中所有词构成的词向量矩阵
    return: result_list 返回拼接而成的一维词向量
    '''
    # 由于之前抽取的特征并没有考虑词与词之间交互对模型的影响，对于分类模型来说，贡献最大的可能是句子中的一部分， 如短语、词组等等。 
    # 用大小不同的滑动窗口(k=[2, 3, 4])， 然后进行平均或取最大操作。
    length = embedding_matrix.shape[0]
    if length>3:
        maxline2 = store(embedding_matrix,2)
        maxline3 = store(embedding_matrix,3)
        maxline4 = store(embedding_matrix,4) 
    elif length==3:
        maxline2 = store(embedding_matrix,2)
        maxline3 = store(embedding_matrix,3)
        maxline4 = maxline3
    elif length==2:
        maxline2 = store(embedding_matrix,2)
        maxline3 = maxline2
        maxline4 = maxline2
    else:
        rl = np.hstack((np.hstack((embedding_matrix, embedding_matrix)),
                     embedding_matrix))
        result_list = rl.reshape(900,)
        return result_list
        
    result_list = np.hstack((np.hstack((maxline2, maxline3)),
                     maxline4))
    
    return result_list

In [ ]:
def Find_Label_embedding(word_matrix, label_embedding):
    '''
    函数说明：获取到所有类别的 label embedding， 与输入的 word embedding 矩阵相乘， 对其结果进行 softmax 运算，
            对 attention score 与输入的 word embedding 相乘的结果求平均或者取最大
    return: (np.array 1D) the embedding by join label and word
    '''
    global train
    label_emb = np.array([fast_embedding.wv.get_vector(s) for s in train['label'].unique()])
    label_embedding = label_emb
    # 第一步：基于consin相似度计算word embedding向量与label embedding之间的相似度
    cosine = np.zeros((word_matrix.shape[0],label_embedding.shape[0]))
    for i in range(word_matrix.shape[0]):
        for j in range(label_embedding.shape[0]):
            cosine[i,j]=cal_cos(word_matrix[i],label_embedding[j])
    
    # 第二步：通过softmax获取注意力分布
    softmax = softmax_cal(cosine)
    
    # 第三步：将求得到的注意力分布与输入的word embedding相乘，并对结果进行最大化或求平均
    mul = np.dot(word_matrix.T,softmax)
    result_embedding = np.average(mul, axis=1)
    
    return result_embedding

In [ ]:
def sentence2vec(query):
    '''
    函数说明：联合多种特征工程来构造新的样本表示，主要通过以下三种特征工程方法
            第一：利用word-embedding的average pooling和max-pooling
            第二：利用窗口size=2，3，4对word-embedding进行卷积操作，然后再进行max/avg-pooling操作
            第二：利用类别标签的表示，增加了词语和标签之间的语义交互，以此达到对词级别语义信息更深层次的考虑
            另外，对于词向量超过预定义的长度则进行截断，小于则进行填充
    参数说明：
    - query:数据集中的每一个样本
    return: 返回样本经过哦特征工程之后得到的词向量
    '''
    global max_length
    arr = []
    # 加载fast_embedding,w2v_embedding
    global fast_embedding, w2v_embedding
    fast_arr = np.array([fast_embedding.wv.get_vector(s)
                         for s in query if s in fast_embedding.wv.vocab.keys()])
    # 在fast_arr下滑动获取到的词向量
    if len(fast_arr) > 0:
        windows_fastarr = np.array(Find_embedding_with_windows(fast_arr))
        result_attention_embedding = Find_Label_embedding(
            fast_arr, fast_embedding)
    else:# 如果样本中的词都不在字典，则该词向量初始化为0
        # 这里300表示训练词嵌入设置的维度为300
        windows_fastarr = np.zeros(300) 
        result_attention_embedding = np.zeros(300)

    fast_arr_max = np.max(np.array(fast_arr), axis=0) if len(
        fast_arr) > 0 else np.zeros(300)
    fast_arr_avg = np.mean(np.array(fast_arr), axis=0) if len(
        fast_arr) > 0 else np.zeros(300)

    fast_arr = np.hstack((fast_arr_avg, fast_arr_max))
    # 将多个embedding进行横向拼接
    arr = np.hstack((np.hstack((fast_arr, windows_fastarr)),
                     result_attention_embedding))
    global sentence_max_length
    # 如果样本的维度大于指定的长度则需要进行截取或者拼凑,
    result_arr = arr[:sentence_max_length] if len(arr) > sentence_max_length else np.hstack((
        arr, np.zeros(int(sentence_max_length-len(arr)))))
    return result_arr

In [ ]:
def Dimension_Reduction(Train, Test):
    '''
    函数说明：该函数通过PCA算法对样本进行降维。
    参数说明：
    - Train: 表示训练数据集
    - Test: 表示测试数据集
    Return: 返回降维之后的数据样本
    '''
    global max_length
    pca = PCA(n_components=max_length)
    pca_train = pca.fit_transform(Train)
    pca_test = pca.fit_transform(Test)
    return pca_train, pca_test

In [ ]:
def Find_Embedding():
    '''
    函数说明：该函数用于获取经过特征工程之后的样本表示
    Return:训练集特征数组(2D)，测试集特征数组(2D)，训练集标签数组（1D）,测试集标签数组（1D）
    '''
    print("获取样本表示中...")
    min_max_scaler = preprocessing.MinMaxScaler()
    Train_features2 = min_max_scaler.fit_transform(
        np.vstack(train["queryCutRMStopWord"].apply(sentence2vec)))
    Test_features2 = min_max_scaler.fit_transform(
        np.vstack(test["queryCutRMStopWord"].apply(sentence2vec)))
    print("获取样本词表示完成")
    # 通过PCA对样本表示进行降维
    # Train_features2, Test_features2 = Dimension_Reduction(Train=Train_features2, Test=Test_features2)
    Train_label2 = train["labelIndex"]
    Test_label2 = test["labelIndex"]

    print("加载训练好的词向量")
    print("Train_features.shape =", Train_features2.shape)
    print("Test_features.shape =", Test_features2.shape)
    print("Train_label.shape =", Train_label2.shape)
    print("Test_label.shape =", Test_label2.shape)

    return Train_features2, Test_features2, Train_label2, Test_label2


In [ ]:
def Predict(Train_label, Test_label, Train_predict_label, Test_predict_label, model_name):
    '''
    函数说明：直接输出训练集和测试在模型上的准确率
    参数说明：
        - Train_label: 真实的训练集标签（1D）
        - Test_labelb: 真实的测试集标签（1D）
        - Train_predict_label: 模型在训练集上的预测的标签(1D)
        - Test_predict_label: 模型在测试集上的预测标签（1D）
        - model_name: 表示训练好的模型
    Return: None
    '''
    # 通过调用metrics.accuracy_score计算训练集和测试集上的准确率
    acc_train = metrics.accuracy_score(Train_label,Train_predict_label)
    #print(Search_Flag+model_name+'_'+'Train accuracy %s' % (acc_train))
    print(model_name+'_'+'Train accuracy %s' % (acc_train))
    # 测试集的准确率
    acc_test = metrics.accuracy_score(Test_label,Test_predict_label)
    #print(Search_Flag+model_name+'_'+'test accuracy %s' % (acc_test))
    print(model_name+'_'+'Test accuracy %s' % (acc_test))

In [ ]:
def Grid_Train_model(Train_features, Test_features, Train_label, Test_label):
    '''
    函数说明：基于网格搜索优化的方法搜索模型最优参数，最后保存训练好的模型
    参数说明：
        - Train_features: 训练集特征数组（2D）
        - Test_features: 测试集特征数组（2D）
        - Train_label: 真实的训练集标签 (1D)
        - Test_label: 真实的测试集标签（1D）
    Return: None
    '''
    models = [
        lgb.LGBMClassifier(max_depth=3, 
                           num_leaves=30,
                           learning_rate=0.05, 
                           n_estimators=1000, 
                           min_child_weight=2, 
                           max_delta_step=0.2, 
                           subsample=0.8, #bagging_fraction
                           bagging_freq= 8,
                           colsample_bytree=0.8, # feature_fraction
                           reg_alpha=1, 
                           reg_lambda=10, 
                           scale_pos_weight=0.2,
                           device="gpu",
                           max_bin=63)
    ]
    # 遍历模型
    for model in models:
        model_name = model.__class__.  __name__
        #gsearch = GridSearchCV(model, param_grid=parameters, scoring='accuracy', cv=3, n_jobs=-1)
        #gsearch.fit(Train_features, Train_label)
        model.fit(Train_features, Train_label)
        # 输出最好的参数
        #print("Best parameters set found on development set:{}".format(
            #gsearch.best_params_))
        #Test_predict_label = gsearch.predict(Test_features)
        #Train_predict_label = gsearch.predict(Train_features)
        #Predict(Train_label, Test_label,
                #Train_predict_label, Test_predict_label, model_name)
        Test_predict_label = model.predict(Test_features)
        Train_predict_label = model.predict(Train_features)
        # print(Test_predict_label)
        Predict(Train_label, Test_label,
                Train_predict_label, Test_predict_label, model_name)


    # 保存训练好的模型
    joblib.dump(model, 'LightGBM.pkl')


主函数,先求训练集和测试集的词向量，然后根据Grid搜索来找到最佳参数的分类模型

In [ ]:
Train_features, Test_features, Train_label, Test_label = Find_Embedding()
Grid_Train_model(Train_features, Test_features,Train_label, Test_label)

注：实际运行发现并不用降维，降维没有增速太多但是准确率大幅下降